# metabolome analysis with GNPS

In [1]:
import calour as ca

In [2]:
ca.set_log_level('INFO')

In [7]:
# dat=ca.io.read_gnps_ms('test/bucket_x_xB_test2.csv','test/map_x_xB.txt','test/bucket_x_xB_test2_gnps_file.tsv',normalize=None)
dat=ca.io.read_gnps_ms('./bucket-gnps.tsv','test/map_x_xB.txt','gnps-output.tsv',normalize=None)

2017-10-16 15:37:50 INFO loaded 511 observations, 71  samples
2017-10-16 15:37:50 WARNING the samples are dropped because they have metadata but do not have data: {'cli_4977_2xB_BA2_01_1660', 'cli_4977_32xB_BB4_01_1670', 'cli_4977_14xB_BA6_01_1664', 'cli_4977_17xB_BA7_01_1665', 'cli_4977_5xB_BA3_01_1661', 'cli_4977_26xB_BB2_01_1668', 'cli_4977_29xB_BB3_01_1669', 'cli_4977_8xB_BA4_01_1662', 'cli_4977_11xB_BA5_01_1663', 'cli_4977_20xB_BA8_01_1666', 'cli_4977_23xB_BB1_01_1667'}
2017-10-16 15:37:50 WARNING the samples have data but do not have metadata: {'cli_4977_11z_BD3_01_1601', 'cli_4977_18x_RA1_01_1616', 'cli_4977_8y_BB7_01_1588', 'cli_4977_21y_RB5_01_1629', 'cli_4977_20z_BD6_01_1604', 'cli_4977_11y_BB8_01_1589', 'cli_4977_26z_BD8_01_1606', 'cli_4977_9z_RC4_01_1637', 'cli_4977_27y_RB7_01_1631', 'cli_4977_23z_BD7_01_1605', 'cli_4977_18z_RC7_01_1640', 'cli_4977_20y_BC3_01_1592', 'cli_4977_27z_RD2_01_1643', 'cli_4977_29z_BE1_01_1607', 'cli_4977_14y_BC1_01_1590', 'cli_4977_33z_RD4_01_1645

In [3]:
# dat=ca.io.read_gnps_ms('./bucket-gnps.tsv','map.txt',normalize=None)
dat=ca.io.read_gnps_ms('./bucket-gnps.tsv','map.txt','./gnps-output.tsv',normalize=None)
# dat=ca.read_open_ms('table2.csv','map.txt',rows_are_samples=True,normalize=None)
# dat=ca.read_open_ms('table2.csv','map.txt','./gnps-output.tsv',rows_are_samples=True,normalize=None)

2017-10-16 15:24:57 INFO loaded 511 observations, 71  samples


In [8]:
datc=dat.cluster_features(1)

2017-10-16 15:50:43 INFO 511 remaining


In [9]:
datc=datc.sort_samples('time_days')

In [10]:
datc=datc.sort_samples('batch')

In [11]:
datc.plot_sort('group',gui='qt5',sample_color_bars=['batch'])

In [12]:
dd=datc.diff_abundance('batch',1,2)

2017-09-27 12:50:23 INFO 68 samples with both values
2017-09-27 12:50:23 INFO 507 remaining
2017-09-27 12:50:23 INFO 33 samples with value 1 ([1])
2017-09-27 12:50:23 INFO method meandiff. number of higher in [1] : 4. number of higher in [2] : 1. total 5


In [13]:
dd.plot_sort('group',gui='qt5',sample_color_bars=['batch'])

2017-09-27 12:50:32 WARNING Cannot initialize GNPS database since gnps info file was not supplied. Please supply when loading


In [16]:
dd=datc.diff_abundance('group','x','z')

2017-09-27 12:34:28 INFO 44 samples with both values
2017-09-27 12:34:28 INFO 453 remaining
2017-09-27 12:34:28 INFO 23 samples with value 1 (['x'])
2017-09-27 12:34:28 INFO method meandiff. number of higher in ['x'] : 26. number of higher in ['z'] : 18. total 44


In [17]:
dd.plot_sort('group',gui='qt5')

2017-09-27 12:34:34 WARNING Cannot initialize GNPS database since gnps info file was not supplied. Please supply when loading


In [18]:
tt=datc.filter_samples('time_days',[16,21,25,30])

In [19]:
dd=tt.diff_abundance('group','x','z')

2017-09-27 12:40:12 INFO 16 samples with both values
2017-09-27 12:40:12 INFO 315 remaining
2017-09-27 12:40:12 INFO 8 samples with value 1 (['x'])
2017-09-27 12:40:12 INFO method meandiff. number of higher in ['x'] : 18. number of higher in ['z'] : 7. total 25


In [20]:
zz=datc.filter_ids(dd.feature_metadata.index)

In [21]:
zz.plot_sort('group',gui='qt5')

2017-09-27 12:40:34 WARNING Cannot initialize GNPS database since gnps info file was not supplied. Please supply when loading
